In [1]:
import numpy as np
import sqlite3
import pandas as pd
import csv
import string
from collections import Counter

In [2]:
data = pd.read_csv("joined_data.csv")

In [4]:
data.Codes.apply(lambda x: len(x)).max()

24914

In [23]:
# open the database
#PORT = 3306
#connection = MySQLdb.connect(port=PORT, user='root', passwd='', db='sherpa')
#cursor = connection.cursor()

path_to_db = "../disease_progression/data/ihc.db"
connection = sqlite3.connect(path_to_db)
cursor = connection.cursor()
# get code and date sequences
#q = "SET SESSION group_concat_max_len = 1000000;" # allow for really long code sequences, otherwise it cuts off.
#cursor.execute(q)

q = """
    select ID,
    group_concat(ICD) Codes, 
    group_concat(substr(DATE,0,11)) Dates 
from claims
where ICD != ''
group by ID;
"""
cursor.execute(q)
# write results to csv
with open('code_sequences.csv', 'wb') as csvfile:
    codewriter = csv.writer(csvfile, delimiter=',')
    for s in cursor:
        #codewriter.writerow([int(s[0]), ",".join([string.strip(w.translate(None,".")) for w in s[1].split(",")]), s[2]])
        codewriter.writerow([int(s[0]), s[1], s[2]])
# close the database
cursor.close()
connection.close()

In [26]:
# open the database
#PORT = 3306
#connection = MySQLdb.connect(port=PORT, user='root', passwd='', db='sherpa')
#cursor = connection.cursor()

path_to_db = "../disease_progression/data/ihc.db"
connection = sqlite3.connect(path_to_db)
cursor = connection.cursor()
# get age and sex
q = "select ID, Birth, Sex from member;"
cursor.execute(q)
#cursor.fetchone() # get rid of the first row, which is blank
# write results to csv
with open('age_sex.csv', 'wb') as csvfile:
    codewriter = csv.writer(csvfile, delimiter=',')
    for s in cursor:
        codewriter.writerow([int(s[0]), s[1], s[2]])
# close the database
cursor.close()
connection.close()

In [27]:
# load up datasets, join together, and save back
codes = pd.read_csv("code_sequences.csv", header=None)
codes.columns = ["ID", "Codes", "Dates"]
demographics = pd.read_csv("age_sex.csv", header=None)
demographics.columns = ["ID", "Birth", "Sex"]
data = pd.merge(demographics, codes, on="ID")
data.to_csv("joined_data.csv")

In [28]:
data.head()

,ID,Birth,Sex,Codes,Dates
0,567622703,2013-08-28 00:00:00,F,"V0381,V0489,7746,6910,7742,7746,V0382,V202,V05...","2014-12-10,2014-01-08,2013-08-29,2014-08-26,20..."
1,564960533,2012-12-31 00:00:00,F,"V202,78095,38200,V802,37555,78060,V2031,5990,V...","2014-01-08,2013-12-16,2014-07-23,2013-01-18,20..."
2,565004294,2012-12-31 00:00:00,F,"38200,V202,V202,V202,38101,V202,7746,V3000,V30...","2013-11-16,2013-10-01,2013-05-01,2013-07-01,20..."
3,565010603,2012-12-31 00:00:00,M,"V053,V068,V202,38200,V0389,V202,V0389,V054,708...","2014-07-14,2013-07-22,2013-10-22,2013-10-22,20..."
4,565121318,2012-12-31 00:00:00,F,"V202,7746,46619,7746,7862,V202,76520,V773,7652...","2014-07-12,2013-01-03,2013-02-20,2013-01-02,20..."


In [79]:
# helper functions to identify codes of interest
# CKD
ckd_filter_string = "substr({}, 1, 3) in ('403', '404', '582', '583', '585', '586') or {} = '5880'"
def checkCKD(c):
    return c[:3] in ('403', '404', '582', '583', '585', '586') or c == '5880'

# Type II Diabetes
diab_filter_string = "substr({}, 1, 3) = '250' and length(ICD) = 5 and substr({}, 5, 5) in ('0', '2')"
def checkDM(c):
    return len(c)==5 and c[:3] =='250' and c[4] in ('0', '2')

In [22]:
s = data.iloc[0,3]

In [23]:
s

'V72.3 ,V72.3 ,V72.3 ,786.05,786.05,786.05,786.05,786.2 ,786.2 '

In [28]:
# process code sequences a bit
# I interrupted this. Start again from i=242761
for i in xrange(data.shape[0]):
    s = data.iloc[i,3]
    data.iloc[i,3] = ",".join([string.strip(w.translate(None,".")) for w in s.split(",")])

KeyboardInterrupt: 

In [29]:
i

242761

In [80]:
# get additional data
# terminated early. Got to index 984.
time_window = 365 # can adjust to 18 months (in days)
day = np.timedelta64(1,"D")
year = np.timedelta64(365,"D")
data["FinalDate"] = ""
data["Diseased"] = 0
data["Span"] = 0
data["ObsCodes"] = ""
data["ObsDate"] = ""
data["Age"] = 0
data["FollowUp"] = 0
count = 0
for i in data.index:
    if count % 1000 == 0:
        print count
    count +=1
    codes = data.loc[i,"Codes"].split(",")
    dates = data.loc[i,"Dates"].split(",")
    ind = -1
    for j,c in enumerate(codes):
        if checkCKD(c):
            ind = j
            data.loc[i,"Diseased"] = 1
            break
    data.loc[i,"FinalDate"] = dates[ind]
    data.loc[i,"Span"] = (np.datetime64(dates[ind])-np.datetime64(dates[0]))/day
    followup = np.random.randint(1,high=time_window)
    obs_date = np.datetime64(dates[ind]) - np.timedelta64(followup, 'D')
    data.loc[i,"Age"] = (obs_date - np.datetime64(data.loc[i,"Birth"]))/year
    data.loc[i,"ObsDate"] = str(obs_date)
    for j,d in enumerate(dates):
        if np.datetime64(d)>obs_date:
            break
    data.loc[i,"ObsCodes"] = ",".join(codes[:j-1])
    data.loc[i,"FollowUp"] = followup
data.Sex = (data.Sex=='F')*1

# save data
data.to_csv("diabetes_data.csv")
data = data.loc[(data.Span-data.FollowUp>30),["Gender","Age","ObsCodes","FollowUp", "Diseased"]]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [81]:
data.to_csv("diabetes_data.csv")
data = data.loc[(data.Span-data.FollowUp>30),["Gender","Age","ObsCodes","FollowUp", "Diseased"]]

In [83]:
data[data.Diseased==1].shape

(2325, 5)

In [51]:
for c,d in zip(data.loc[i,"Codes"].split(","), data.loc[i,"Dates"].split(",")):
    print c, d

7862 2007-10-18
7862 2007-10-18
7862 2009-04-22
462 2009-04-22
462 2009-04-22
7862 2009-04-22
462 2009-04-22
462 2009-04-22
462 2009-04-22
462 2009-04-22
462 2009-04-22
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
51881 2009-04-24
0382 2009-04-24
51881 2009-04-24
0382 2009-04-24
79902 2009-04-24
0382 2009-04-24
7862 2009-04-24
5119 2009-04-24
0382 2009-04-24
2888 2009-04-24
5609 2009-04-24
5119 2009-04-24
78605 2009-04-24
78650 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
0382 2009-04-24
51881 2009-04-25
5849 2009-04-25
51881 2009-04-25
78600 2009-04-25
51881 2

In [53]:
v = data.Codes.apply(lambda x: len(x.split(",")))

In [57]:
u = data.Dates.apply(lambda x: len(x.split(",")))